In [1]:
using MCMachineLearningPotentialInterface

In [2]:
using BenchmarkTools,DelimitedFiles

The target potential is the Embedded Atom Model:
$$
    U_{ref} = \frac{1}{2}\sum_{i\neq j} \epsilon\left(\frac{a}{r_{ij}}\right)^n  - \epsilon C \sum_i \left[\sum_{j\neq i}\left(\frac{a}{r_{ij}} \right)^m  \right]^{1/2}
$$



We want to split this into a few components, for example, we could first calculate $E_i$ and then sum the terms. We can also relate each of the vectors $\frac{a}{r_{ij}}$ to the dist2_vec universal to our monte carlo simulations. aka something like sum(a/disvec .^m ) Maybe a second look at how the LJ potential is managed? Maybe consider how to calculate the first instance then consider how subsequent calculations are managed

In [3]:
n = 8.482
m = 4.692
ϵ = 0.1359725
a = 4.72432
C = 27.561
ean = ϵ*a^n
ecm = ϵ*C*sqrt(a^m)

143.13507411588827

In [4]:
ico_55 = [[0.0000006584,       -0.0000019175,        0.0000000505],
[-0.0000005810,       -0.0000004871,        0.6678432175],
[0.1845874248,       -0.5681026047,        0.2986701538],
[-0.4832557457,       -0.3511072166,        0.2986684497],
[-0.4832557570,        0.3511046452,        0.2986669456],
[0.1845874064,        0.5681000550,        0.2986677202],
[0.5973371920,       -0.0000012681,        0.2986697030],
[-0.1845860897,       -0.5681038901,       -0.2986676192],
[-0.5973358752,       -0.0000025669,       -0.2986696020],
[-0.1845861081,        0.5680987696,       -0.2986700528],
[0.4832570624,        0.3511033815,       -0.2986683486],
[0.4832570738,       -0.3511084803,       -0.2986668445],
[0.0000018978,       -0.0000033480,       -0.6678431165],
[-0.0000017969,        0.0000009162,        1.3230014650],
[0.1871182835,       -0.5758942175,        0.9797717078],
[-0.4898861924,       -0.3559221410,       0.9797699802],
[-0.4898862039,        0.3559224872,        0.9797684555],
[0.1871182648,        0.5758945856,        0.9797692407],
[0.6055300485,        0.0000001908,        0.9797712507],
[0.7926501864,       -0.5758950093,        0.6055339635],
[0.3656681761,       -1.1254128670,        0.5916673591],
[-0.3027660545,       -0.9318173412,        0.6055326929],
[-0.9573332453,       -0.6955436707,        0.5916639831],
[-0.9797705418,       -0.0000006364,        0.6055294407],
[-0.9573332679,        0.6955423392,        0.5916610035],
[-0.3027660847,        0.9318160902,        0.6055287012],
[0.3656681396,        1.1254115783,        0.5916625380],
[0.7926501677,        0.5758937939,        0.6055314964],
[1.1833279992,       -0.0000006311,        0.5916664660],
[0.6770051458,       -0.9318186223,        0.0000033028],
[0.0000006771,       -1.1517907207,        0.0000025175],
[-0.6770037988,       -0.9318186442,        0.0000007900],
[-1.0954155825,       -0.3559242494,       -0.0000012200],
[-1.0954155940,        0.3559203788,       -0.0000027447],
[-0.6770038290,        0.9318147872,       -0.0000032017],
[0.0000006397,        1.1517868856,       -0.0000024165],
[0.6770051155,        0.9318148091,       -0.0000006889],
[1.0954168993,        0.3559204143,        0.0000013211],
[1.0954169108,       -0.3559242139,        0.0000028458],
[0.3027674014,       -0.9318199253,       -0.6055286002],
[-0.3656668229,       -1.1254154134,       -0.5916624370],
[-0.7926488510,       -0.5758976290,       -0.6055313954],
[-1.1833266824,       -0.0000032040,       -0.5916663649],
[-0.7926488697,        0.5758911742,       -0.6055338624],
[-0.3656668594,        1.1254090319,       -0.5916672580],
[0.3027673712,        0.9318135061,       -0.6055325919],
[0.9573345621,        0.6955398357,       -0.5916638820],
[0.9797718586,       -0.0000031986,       -0.6055293396],
[0.9573345846,       -0.6955461743,       -0.5916609025],
[-0.1871169480,       -0.5758984207,       -0.9797691397],
[-0.6055287318,       -0.0000040259,       -0.9797711497],
[-0.1871169667,        0.5758903824,       -0.9797716067],
[0.4898875091,        0.3559183059,       -0.9797698792],
[0.4898875207,       -0.3559263223,       -0.9797683545],
[0.0000031136,       -0.0000047513,       -1.3230013639]]
#convert to Bohr
nmtobohr = 18.8973
copperconstant = 0.36258*nmtobohr
pos_cu55 = copperconstant*ico_55


55-element Vector{Vector{Float64}}:
 [4.5112139495856e-6, -1.3138293967695e-5, 3.4601504321700004e-7]
 [-3.980885942754e-6, -3.3375035158614e-6, 4.575916827038472]
 [1.264752985534391, -3.8925157884546686, 2.0464230925690106]
 [-3.311163519470279, -2.405710469814843, 2.0464114164455425]
 [-3.311163596895427, 2.40569285113995, 2.0464011106786812]
 [1.264752859461583, 3.8924983184634674, 2.046406418069819]
 [4.092824837722801, -8.6887460654154e-6, 2.046420003785219]
 [-1.2647438377188622, -3.8925245957365804, -2.046405726039733]
 [-4.092815815294902, -1.75878418699746e-5, -2.0464193117551326]
 [-1.26474396379167, 3.8924895111815556, -2.046422400538924]
 ⋮
 [6.5594487104586, 4.765688045720408, -4.053952548518178]
 [6.713184197946128, -2.1916113212552402e-5, -4.1489556556605045]
 [6.559448864623718, -4.765731476432347, -4.053932133630629]
 [-1.2820847296802604, -3.9459310282596545, -6.7131655686332365]
 [-4.148951491146777, -2.75845933165806e-5, -6.713179340717135]
 [-1.282084857808603, 3.

In [6]:
dist2_mat = find_distance2_mat(pos_cu55)

55×55 Matrix{Float64}:
  0.0      20.939    20.939    20.939   …   62.2806   62.2806   82.1728
 20.939     0.0      23.1496   23.1496     144.657   144.657   186.072
 20.939    23.1496    0.0      23.1496     121.19     83.2196  140.213
 20.939    23.1496   23.1496    0.0        144.657   121.19    140.213
 20.939    23.1496   60.6064   23.1496     121.19    144.657   140.213
 20.939    23.1496   60.6064   60.6064  …   83.2196  121.19    140.213
 20.939    23.1496   23.1496   60.6064      83.2196   83.2196  140.213
 20.939    60.6064   23.1496   23.1496      83.2196   45.249    66.0106
 20.939    60.6064   60.6064   23.1496      83.2196   83.2196   66.0106
 20.939    60.6064   83.756    60.6064      45.249    83.2196   66.0106
  ⋮                                     ⋱                      
 82.1728  140.213   140.213   186.072       22.7447   69.2332   90.848
 62.2806  121.19     83.2196  144.657       23.7891   23.7891   69.2332
 82.1728  140.213    66.0106  140.213       69.2332   22